In [1]:
def load_words(filename):
    with open(filename, 'r') as file:
        words = file.read().splitlines()
    return words

words = load_words('../words_alpha.txt')

In [2]:
import re

def count_syllables(word):
    word = word.lower()
    # Find vowel groups
    syllable_count = len(re.findall(r'[aeiouy]+', word))
    # Adjust for silent 'e'
    if word.endswith('e'):
        syllable_count -= 1
    # Ensure at least 1 syllable
    return max(1, syllable_count)

# Apply the heuristic
syllable_data = [(word, count_syllables(word)) for word in words]

In [ ]:
import pyodbc as odbc
import os
from dotenv import load_dotenv

load_dotenv()
SERVER = os.getenv('SERVER')
DATABASE = os.getenv('DATABASE')
# PASS = os.getenv('PASSWORD')
# USER = os.getenv('USER')

connectionString = f"""
DRIVER={{SQL Server}};
SERVER={SERVER};
DATABASE={DATABASE};
Trusted_Connection=yes;
"""
# pwd={PASS};
# uid={USER};
# Connect to SQL Server database
conn = odbc.connect(connectionString)
print(conn)